In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santander-customer-satisfaction/sample_submission.csv
/kaggle/input/santander-customer-satisfaction/train.csv
/kaggle/input/santander-customer-satisfaction/test.csv


### 1. Preparação
Dado a performance do dataset de Undersampling e do uso do VotingClassifier, dessa vez, tentaremos otimizar cada classificador individualmente no Undersampling antes de fazer uso do VotingClassifier com todos.

In [6]:
dataset = pd.read_csv("/kaggle/input/santander-customer-satisfaction/train.csv")
COLUNAS_CONSTANTES = [coluna for coluna in dataset.columns if dataset[coluna].nunique() == 1]
dataset = dataset.drop(columns = COLUNAS_CONSTANTES)

In [7]:
ID = dataset.ID
dataset.drop(columns = "ID", inplace = True)
TARGET = dataset.TARGET
dataset.drop(columns = "TARGET", inplace = True)

In [8]:
dataset.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016


### 1.1 Pré-Processamento

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

dataset_scaled = scaler.fit_transform(dataset.copy())

dataset_scaled = pd.DataFrame(dataset_scaled, columns = dataset.columns)
dataset_scaled.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.18,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001545
1,0.999764,0.29,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002003
2,0.999764,0.18,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002822
3,0.999764,0.32,0.0,0.01513,0.009275,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002671
4,0.999764,0.34,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091


In [10]:
from imblearn.under_sampling import RandomUnderSampler

SEED = 912039

In [11]:
sampler = RandomUnderSampler(random_state = SEED)
x_under, y_under = sampler.fit_resample(dataset_scaled, TARGET)
dataset_under = pd.concat([x_under, y_under], axis = 1)
dataset_under.TARGET.value_counts()

TARGET
0    3008
1    3008
Name: count, dtype: int64

In [12]:
y_undersampling = dataset_under.TARGET
dataset_under.drop("TARGET", axis = 1, inplace = True)

### 2. Treinamento

In [13]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

linear = SVC(random_state = SEED)
knn = KNeighborsClassifier()
lr = LogisticRegression(random_state = SEED)
dt = DecisionTreeClassifier(random_state = SEED)
rf = RandomForestClassifier(random_state = SEED)
ab = AdaBoostClassifier(random_state = SEED)
gb = GradientBoostingClassifier(random_state = SEED)
extra = ExtraTreesClassifier(random_state = SEED)
mlp = MLPClassifier(random_state = SEED)

In [14]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, roc_auc_score

def avalia_modelo(modelo, x, y):
    scoring = {
        'accuracy': 'accuracy',
        'precision': 'precision',
        'recall': 'recall',
        'f1': 'f1',
        'roc_auc': make_scorer(roc_auc_score)
    }
    
    resultados = cross_validate(modelo, x, y, scoring=scoring, cv=10, n_jobs=-1)
    
    print(f'Modelo: {modelo}')
    media_acuracia = sum(resultados['test_accuracy'])/10
    media_precisao = sum(resultados['test_precision'])/10
    media_recall = sum(resultados['test_recall'])/10
    media_f1 = sum(resultados['test_f1'])/10
    media_roc_auc = sum(resultados['test_roc_auc'])/10
    
    print(f'Acurácia --> {media_acuracia :.2f}')
    print(f'Precisão --> {media_precisao :.2f}')
    print(f'Recall --> {media_recall :.2f}')
    print(f'F1 --> {media_f1 :.2f}')
    print(f'ROC AUC --> {media_roc_auc :.2f}')

#### 2.1 KNN

In [16]:
avalia_modelo(knn, dataset_under, y_undersampling)

Modelo: KNeighborsClassifier()
Acurácia --> 0.69
Precisão --> 0.69
Recall --> 0.68
F1 --> 0.68
ROC AUC --> 0.69


In [17]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [24]:
lista_parametros = {
    'n_neighbors': np.arange(3, 30),
    'weights': ['uniform', 'distance'],
    'p': np.arange(2, 5)
}

busca = RandomizedSearchCV(KNeighborsClassifier(), lista_parametros,
                           cv = 5, n_iter = 30, random_state = SEED,
                           scoring = "accuracy", n_jobs = -1)
busca.fit(dataset_under, y_undersampling)
print(busca.best_estimator_)
print(busca.best_score_ * 100)

KNeighborsClassifier(n_neighbors=19, p=3)
72.53966412871475


In [25]:
avalia_modelo(KNeighborsClassifier(n_neighbors=19, p=3), dataset_under, y_undersampling)

Modelo: KNeighborsClassifier(n_neighbors=19, p=3)
Acurácia --> 0.73
Precisão --> 0.74
Recall --> 0.70
F1 --> 0.72
ROC AUC --> 0.73


In [32]:
KNN = KNeighborsClassifier(n_neighbors=19, p=3)

#### 2.2 Regressão Logística

In [22]:
avalia_modelo(lr, dataset_under, y_undersampling)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: LogisticRegression(random_state=912039)
Acurácia --> 0.70
Precisão --> 0.69
Recall --> 0.74
F1 --> 0.71
ROC AUC --> 0.70


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [27]:
lista_parametros = {
    'penalty': ['l1', 'l2'],
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'tol': [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1],
    'max_iter': [100, 200, 300, 400, 500, 1000]
}

busca = RandomizedSearchCV(LogisticRegression(random_state = SEED), lista_parametros,
                           cv = 5, n_iter = 30, random_state = SEED,
                           scoring = "accuracy", n_jobs = -1)
busca.fit(dataset_under, y_undersampling)
print(busca.best_estimator_)
print(busca.best_score_ * 100)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

LogisticRegression(C=1000, max_iter=1000, random_state=912039, tol=0.01)
71.22697133136153


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [35]:
LOGISTICREGRESSION = LogisticRegression(C=1000, max_iter=10000, random_state=912039, tol=0.01)

In [36]:
avalia_modelo(LOGISTICREGRESSION, dataset_under, y_undersampling)

Modelo: LogisticRegression(C=1000, max_iter=10000, random_state=912039, tol=0.01)
Acurácia --> 0.71
Precisão --> 0.70
Recall --> 0.75
F1 --> 0.72
ROC AUC --> 0.71


#### 2.3 SVC

In [37]:
avalia_modelo(linear, dataset_under, y_undersampling)

Modelo: SVC(random_state=912039)
Acurácia --> 0.69
Precisão --> 0.70
Recall --> 0.67
F1 --> 0.68
ROC AUC --> 0.69


In [38]:
lista_parametros = {
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'tol': [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1],
    'gamma': ['scale', 'auto']
}

busca = RandomizedSearchCV(SVC(random_state = SEED), lista_parametros,
                           cv = 5, n_iter = 30, random_state = SEED,
                           scoring = "accuracy", n_jobs = -1)
busca.fit(dataset_under, y_undersampling)
print(busca.best_estimator_)
print(busca.best_score_ * 100)

SVC(C=1000, random_state=912039, tol=1e-06)
72.29074324156389


In [39]:
SVM = SVC(C=1000, random_state=912039, tol=1e-06)

In [40]:
avalia_modelo(SVM, dataset_under, y_undersampling)

Modelo: SVC(C=1000, random_state=912039, tol=1e-06)
Acurácia --> 0.72
Precisão --> 0.73
Recall --> 0.71
F1 --> 0.72
ROC AUC --> 0.72


#### 2.4 RandomForest

In [41]:
avalia_modelo(rf, dataset_under, y_undersampling)

Modelo: RandomForestClassifier(random_state=912039)
Acurácia --> 0.73
Precisão --> 0.75
Recall --> 0.71
F1 --> 0.73
ROC AUC --> 0.73


In [42]:
lista_parametros = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['sqrt', 'log2', None],
    'min_samples_split':[2, 5, 10],
    'criterion': ['gini', 'entropy', 'log_loss']
}

busca = RandomizedSearchCV(RandomForestClassifier(random_state = SEED), lista_parametros,
                           cv = 5, n_iter = 30, random_state = SEED,
                           scoring = "accuracy", n_jobs = -1)
busca.fit(dataset_under, y_undersampling)
print(busca.best_estimator_)
print(busca.best_score_ * 100)

RandomForestClassifier(criterion='log_loss', min_samples_split=10,
                       n_estimators=200, random_state=912039)
75.49884977478783


In [43]:
BAGGING = RandomForestClassifier(criterion='log_loss', min_samples_split=10,
                       n_estimators=200, random_state=912039)

In [44]:
avalia_modelo(BAGGING, dataset_under, y_undersampling)

Modelo: RandomForestClassifier(criterion='log_loss', min_samples_split=10,
                       n_estimators=200, random_state=912039)
Acurácia --> 0.75
Precisão --> 0.76
Recall --> 0.74
F1 --> 0.75
ROC AUC --> 0.75


#### 2.5 GradientBoosting

In [45]:
avalia_modelo(gb, dataset_under, y_undersampling)

Modelo: GradientBoostingClassifier(random_state=912039)
Acurácia --> 0.76
Precisão --> 0.77
Recall --> 0.76
F1 --> 0.76
ROC AUC --> 0.76


In [46]:
lista_parametros = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['sqrt', 'log2', None],
    'criterion':['friedman_mse', 'squared_error'],
    'loss': ['exponential', 'log_loss'],
    'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

busca = RandomizedSearchCV(GradientBoostingClassifier(random_state = SEED), lista_parametros,
                           cv = 5, n_iter = 30, random_state = SEED,
                           scoring = "accuracy", n_jobs = -1)
busca.fit(dataset_under, y_undersampling)
print(busca.best_estimator_)
print(busca.best_score_ * 100)

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_gb_losses.py:924: RuntimeWarning: overflow encountered in exp
  * np.sum(sample_weight * np.exp(-(2 * y - 1) * raw_predictions))
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_gb_losses.py:940: RuntimeWarning: overflow encountered in exp
  return y_ * np.exp(-y_ * raw_predictions.ravel())
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_gb_losses.py:960: RuntimeWarning: overflow encountered in exp
  numerator = np.sum(y_ * sample_weight * np.exp(-y_ * raw_predictions))
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_gb_losses.py:961: RuntimeWarning: overflow encountered in exp
  denominator = np.sum(sample_weight * np.exp(-y_ * raw_predictions))
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_gb_losses.py:967: RuntimeWarning: invalid value encountered in double_scalars
  tree.value[leaf, 0, 0] = numerator / denominator
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_gb_losses.py:

GradientBoostingClassifier(learning_rate=0.01, loss='exponential',
                           n_estimators=400, random_state=912039)
76.11388196176227


In [47]:
BOOSTING = GradientBoostingClassifier(learning_rate=0.01, loss='exponential',
                           n_estimators=400, random_state=912039)

In [48]:
avalia_modelo(BOOSTING, dataset_under, y_undersampling)

Modelo: GradientBoostingClassifier(learning_rate=0.01, loss='exponential',
                           n_estimators=400, random_state=912039)
Acurácia --> 0.76
Precisão --> 0.77
Recall --> 0.76
F1 --> 0.76
ROC AUC --> 0.76


#### 2.6 MLP

In [49]:
avalia_modelo(mlp, dataset_under, y_undersampling)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptro

Modelo: MLPClassifier(random_state=912039)
Acurácia --> 0.74
Precisão --> 0.74
Recall --> 0.74
F1 --> 0.74
ROC AUC --> 0.74


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [50]:
lista_parametros = {
    'hidden_layer_sizes': [(50, 50), (100, 100), (50, 100, 50)],
    'activation': ['logistic', 'tanh', 'relu'],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'max_iter': [200, 300, 500, 1000],
}

busca = RandomizedSearchCV(MLPClassifier(random_state = SEED), lista_parametros,
                           cv = 5, n_iter = 30, random_state = SEED,
                           scoring = "accuracy", n_jobs = -1)
busca.fit(dataset_under, y_undersampling)
print(busca.best_estimator_)
print(busca.best_score_ * 100)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='tanh', hidden_layer_sizes=(50, 50),
              learning_rate='invscaling', max_iter=1000, random_state=912039)
74.4181904043877


In [51]:
REDENEURAL = MLPClassifier(activation='tanh', hidden_layer_sizes=(50, 50),
              learning_rate='invscaling', max_iter=1000, random_state=912039)

In [52]:
avalia_modelo(REDENEURAL, dataset_under, y_undersampling)

Modelo: MLPClassifier(activation='tanh', hidden_layer_sizes=(50, 50),
              learning_rate='invscaling', max_iter=1000, random_state=912039)
Acurácia --> 0.74
Precisão --> 0.76
Recall --> 0.71
F1 --> 0.73
ROC AUC --> 0.74


### 3. VotingClassifier

In [53]:
from sklearn.ensemble import VotingClassifier

vc = VotingClassifier(
    estimators = [
        ("KNeighbors", KNN),
        ("Regressão Logística", LOGISTICREGRESSION),
        ("Máquina de Vetor de Suporte", SVM),
        ("Bagging", BAGGING),
        ("Boosting", BOOSTING),
        ("Rede Neural", REDENEURAL)
    ],
    voting='hard'
)

In [54]:
avalia_modelo(vc, dataset_under, y_undersampling)

Modelo: VotingClassifier(estimators=[('KNeighbors',
                              KNeighborsClassifier(n_neighbors=19, p=3)),
                             ('Regressão Logística',
                              LogisticRegression(C=1000, max_iter=10000,
                                                 random_state=912039,
                                                 tol=0.01)),
                             ('Máquina de Vetor de Suporte',
                              SVC(C=1000, random_state=912039, tol=1e-06)),
                             ('Bagging',
                              RandomForestClassifier(criterion='log_loss',
                                                     min_samples_split=10,
                                                     n_estimators=200,
                                                     random_state=912039)),
                             ('Boosting',
                              GradientBoostingClassifier(learning_rate=0.01,
                       

### 4. Teste

In [55]:
alvo = pd.read_csv("/kaggle/input/santander-customer-satisfaction/test.csv")

alvo_id = alvo.ID
alvo.drop(columns = "ID", inplace = True)

scaler = MinMaxScaler()

copia = alvo.copy()
copia = scaler.fit_transform(copia)

alvo = pd.DataFrame(copia, columns = alvo.columns)
alvo.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001361
1,0.999764,0.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001533
2,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001585
3,0.999764,0.19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.006462
4,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.002507


In [56]:
alvo = alvo.drop(columns = COLUNAS_CONSTANTES)
alvo.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001361
1,0.999764,0.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001533
2,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001585
3,0.999764,0.19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.006462
4,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.002507


In [57]:
vc = VotingClassifier(
    estimators = [
        ("KNeighbors", KNN),
        ("Regressão Logística", LOGISTICREGRESSION),
        ("Máquina de Vetor de Suporte", SVM),
        ("Bagging", BAGGING),
        ("Boosting", BOOSTING),
        ("Rede Neural", REDENEURAL)
    ],
    voting='hard'
)

vc.fit(dataset_under, y_undersampling)

VotingClassifier(estimators=[('KNeighbors',
                              KNeighborsClassifier(n_neighbors=19, p=3)),
                             ('Regressão Logística',
                              LogisticRegression(C=1000, max_iter=10000,
                                                 random_state=912039,
                                                 tol=0.01)),
                             ('Máquina de Vetor de Suporte',
                              SVC(C=1000, random_state=912039, tol=1e-06)),
                             ('Bagging',
                              RandomForestClassifier(criterion='log_loss',
                                                     min_samples_split=10,
                                                     n_estimators=200,
                                                     random_state=912039)),
                             ('Boosting',
                              GradientBoostingClassifier(learning_rate=0.01,
                                                         loss='exponential',
                                                         n_estimators=400,
                                                         random_state=912039)),
                             ('Rede Neural',
                              MLPClassifier(activation='tanh',
                                            hidden_layer_sizes=(50, 50),
                                            learning_rate='invscaling',
                                            max_iter=1000,
                                            random_state=912039))])

In [58]:
resultados = vc.predict(alvo)
resposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)
resposta

,ID,TARGET
0,2,1
1,5,1
2,6,0
3,7,0
4,9,0
...,...,...
75813,151831,1
75814,151832,0
75815,151833,0
75816,151834,0


In [59]:
resposta.TARGET.value_counts()

TARGET
0    53189
1    22629
Name: count, dtype: int64

In [61]:
resposta.to_csv("resposta.csv", index = False)